# Adapter

Benchmarking example

In [ ]:
import time
import pathlib
import csv
import itertools
from datetime import datetime
from tqdm import tqdm

import threading
import pynvml
from utils.adapter import BERTAdapter

In [ ]:
adapter_param_grid = {
    'num_epochs': [50],
    'batch_size': [32, 512],
    'learning_rate': [1e-2, 1e-3],
    'category': [task_config.class_type],
    'device': [device_name]
}

# Create a list of all combinations of hyperparameters
adapter_params = [dict(zip(adapter_param_grid.keys(), v)) for v in itertools.product(*adapter_param_grid.values())]
print(f"{len(adapter_params)} hyperparameter combinations")

In [ ]:
# Setup for saving power outputs
results_folder = pathlib.Path(f"results/adapter/{task_param}")
results_folder.mkdir(parents=True, exist_ok=True)
save_file_id = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = results_folder / f"val_{save_file_id}.csv"
with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['power', 'time'])
print(f"saving results to ./{results_file}")

# Iterate over all combinations of hyperparameters
adapter_bar = tqdm(enumerate(adapter_params), total=len(adapter_params))
for i, params in adapter_bar:
    # Formatting params to display
    print_params = params.copy()
    for param in ['category', 'device']:
        del print_params[param]

    # Initialize the model with current set of hyperparameters
    adapter = BERTAdapter(**params)

    # Create a thread for tracking power output (NVIDIA ONLY), logs power and time
    def get_power():
        pynvml.nvmlInit()
        while not adapter.train_stop_flag:
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # Returns device handle, index 0 specifies the first GPU
            power = pynvml.nvmlDeviceGetPowerUsage(handle)  # Retrieves power usage for GPU in milliwatts
            power = power / 1000  # convert mW to W
            curr_time = time.time()
            print("Current NVIDIA power output:", power, "at time", curr_time)
            # Write to results csv
            with open(results_file, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([power, curr_time])
                
        pynvml.nvmlShutdown()
        
    power_thread = threading.Thread(target=get_power)

    # Make sure power output was recorded in the correct time range
    print("Start time:", time.time()) 

    if recomputed:
        power_thread.start()  # Start the benchmarking thread
        adapter.fit(X_train_recomputed,
                    Y_train,
                    X_val_recomputed,
                    Y_val)
        power_thread.join()  # Wait for the thread to complete
    else:
        power_thread.start()  # Start the benchmarking thread
        adapter.fit(X_train,
                    Y_train,
                    X_val,
                    Y_val)
        power_thread.join()  # Wait for the thread to complete
        
    print("End time:", time.time())